In [1]:
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
from utils import *

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Available devices: ", torch.cuda.device_count())
print("torch version:", torch.__version__)
print("cudnn version:", torch.backends.cudnn.version())
print("cuda version:", torch.version.cuda)

Available devices:  2
torch version: 2.2.0a0+81ea7a4
cudnn version: 8907
cuda version: 12.3


In [3]:
max_length = 1024
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [4]:
pad_tokens = 0
elements = 0
for song in train:
    for e in song:
        if e == '<pad>':
            pad_tokens += 1
        elements += 1

percentage = pad_tokens / elements * 100
print(f"Pad tokens: {pad_tokens}, Percentage: {percentage:.2f}%") 

Pad tokens: 16773876, Percentage: 37.86%


In [5]:
#Convert midi into dtype int
midi_train = midi_train.astype(int)
midi_test = midi_test.astype(int)

In [6]:
print(train.shape, test.shape, midi_train.shape, midi_test.shape)

dataset = TokenDatasetMidi(train, midi_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_test, max_length, tokens)

(43272, 1024) (4800, 1024) (43272, 1024, 8) (4800, 1024, 8)
data has 43272 pieces, 198 unique tokens.
data has 4800 pieces, 198 unique tokens.


In [7]:
stoi = { tk:i for i,tk in enumerate(tokens) }
itos = { i:tk for i,tk in enumerate(tokens) }

print(stoi)

{'.': 0, '/': 1, '0.3997395833333333': 2, '0.4440104166666667': 3, '0.5': 4, '0.5703125': 5, '0.6666666666666666': 6, '0.75': 7, '0.7994791666666666': 8, '0.8880208333333334': 9, '1.0': 10, '1.1419270833333333': 11, '1.3333333333333333': 12, '1.5': 13, '1.5989583333333333': 14, '1.7135416666666667': 15, '128 Feel': 16, '2.0': 17, '2.25': 18, '2.3997395833333335': 19, '2.6666666666666665': 20, '3.0': 21, '4.0': 22, ':|': 23, '<end>': 24, '<pad>': 25, '<start>': 26, '<style>': 27, 'A': 28, 'A major': 29, 'A minor': 30, 'A#': 31, 'A##': 32, 'Ab': 33, 'Ab major': 34, 'Ab minor': 35, 'Abb': 36, 'Afoxé': 37, 'Afro': 38, 'B': 39, 'B major': 40, 'B minor': 41, 'B#': 42, 'B##': 43, 'Baião': 44, 'Ballad': 45, 'Bb': 46, 'Bb major': 47, 'Bb minor': 48, 'Bbb': 49, 'Blues': 50, 'Bolero': 51, 'Bolero-Cha': 52, 'Bossa': 53, 'C': 54, 'C major': 55, 'C minor': 56, 'C#': 57, 'C##': 58, 'Calypso': 59, 'Cb': 60, 'Cbb': 61, 'Cha Cha': 62, 'Chacarera': 63, 'Choro': 64, 'Country Ballad': 65, 'D': 66, 'D major

In [8]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 270,
    }
)

04/18/2024 09:05:31 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


In [9]:
# import subprocess

# # Example command to list processes using GPU (this won't run here due to sandbox restrictions)
# output = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
# print(output.stdout)

In [11]:
epochs = 270
embedding = 512
heads = 4
layers = 4
batch_size = 128
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128
token_size = len(tokens)

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()

04/18/2024 09:06:35 - INFO - model -   number of parameters: 1.283021e+07


../models/model_epochs->270_heads->4_embd->512_batch->128_new_midi_embeddings


04/18/2024 09:06:36 - INFO - model -   number of parameters: 1.283021e+07
epoch 1 iter 338: train loss 2.08555. lr 3.000000e-05: 100%|██████████| 339/339 [02:42<00:00,  2.09it/s]
04/18/2024 09:09:19 - INFO - trainer -   epoch train loss: 2.879668


train loss: 2.8796677329195637


04/18/2024 09:09:27 - INFO - trainer -   test loss: 1.909220
epoch 2 iter 338: train loss 1.57121. lr 3.000000e-05: 100%|██████████| 339/339 [02:40<00:00,  2.11it/s]
04/18/2024 09:12:08 - INFO - trainer -   epoch train loss: 1.865582


...

04/18/2024 13:43:52 - INFO - trainer -   test loss: 0.245850
epoch 100 iter 251: train loss 0.15759. lr 3.000000e-05:  74%|███████▍  | 252/339 [02:03<00:41,  2.10it/s]Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 233, in run
    self._record_writer.write(data)
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 766, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 160, in append
    self._write(filename, file_content, "ab" if binar

KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f08cb3597e0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f0896b716c0, execution_count=11 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 7f0896b717e0, raw_cell="epochs = 270
embedding = 512
heads = 4
layers = 4
.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://attached-container%2B7b22636f6e7461696e65724e616d65223a222f64616c6d617a7a6f5f6368726f6d61666c6f77227d@ssh-remote%2B130.237.3.110/workspace/src/run.ipynb#X12sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

: 